In [8]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, make_scorer, confusion_matrix, recall_score, precision_score
from sklearn.svm import SVC
from scipy.stats import randint
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from imblearn.under_sampling import RandomUnderSampler
import pickle

import os

In [2]:
os.chdir(os.path.dirname(os.getcwd()))

In [26]:
df = pd.read_csv("data/train.csv").reset_index(drop=True)

In [27]:
df.columns

Index(['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type',
       'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver',
       'Android Ver', 'Descargas', 'Descargas2', 'Categoria', 'tamaño',
       'tamaño2', 'Gratuito', 'tipo_contenido', 'Genero1',
       'dias_sin_actualizar', 'meses_sin_actualizar', 'grupomeses',
       'version_actual', 'grupoversiones', 'version_android', 'Genero2',
       'media_inst_grp_ver', 'mediana_inst_grp_ver', 'med_versandroid'],
      dtype='object')

In [ ]:
X = df[['Rating','Categoria', 'tamaño',
       'tamaño2', 'Gratuito', 'tipo_contenido', 'Genero1',
       'dias_sin_actualizar', 'meses_sin_actualizar', 'grupomeses',
       'version_actual', 'grupoversiones', 'version_android', 'Genero2',
       'media_inst_grp_ver', 'mediana_inst_grp_ver', 'med_versandroid']]

y = df['Descargas2']

In [28]:
X = df[['Rating','Categoria', 'tamaño',
       'tamaño2', 'Gratuito', 'tipo_contenido', 'Genero1',
       'dias_sin_actualizar', 'meses_sin_actualizar', 'grupomeses',
       'version_actual', 'grupoversiones', 'version_android', 'Genero2',
       'media_inst_grp_ver', 'mediana_inst_grp_ver', 'med_versandroid']]

y = df['Descargas2']

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42)

In [51]:
final_test = pd.read_csv("data/test.csv")
final_test_x = final_test[['Rating','Categoria', 'tamaño',
       'tamaño2', 'Gratuito', 'tipo_contenido', 'Genero1',
       'dias_sin_actualizar', 'meses_sin_actualizar', 'grupomeses',
       'version_actual', 'grupoversiones', 'version_android', 'Genero2',
       'media_inst_grp_ver', 'mediana_inst_grp_ver', 'med_versandroid']]

final_test_y = final_test['Descargas2']

In [30]:
rus = RandomUnderSampler(sampling_strategy='auto', random_state=42)
X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)


In [31]:
model = RandomForestClassifier()
model.fit(X_train,y_train)
predictions = model.predict(X_test)
print(model.score(X_train,y_train))
print("Accuracy Score:",accuracy_score(y_test, predictions))
print("Precision Score:",precision_score(y_test,predictions, average="macro"))
print("Recall Score:",recall_score(y_test, predictions, average="macro"))
print(confusion_matrix(y_test, predictions))

0.9985935302390999
Accuracy Score: 0.4142053445850914
Precision Score: 0.4453651744236706
Recall Score: 0.4176397179571945
[[123  31  18  10  12   0]
 [ 17  53  59  30  16   6]
 [  6  57  86  65  30  11]
 [  2  19  68  76  75  14]
 [  2  11  45  64 141  54]
 [  1   3   7  17  83 110]]


In [34]:
model2 = RandomForestClassifier()
model2.fit(X_train_resampled,y_train_resampled)
predictions2 = model2.predict(X_test)
print(model2.score(X_train_resampled,y_train_resampled))
print("Accuracy Score:",accuracy_score(y_test, predictions2))
print("Precision Score:",precision_score(y_test,predictions2, average="macro"))
print("Recall Score:",recall_score(y_test, predictions2, average="macro"))
print(confusion_matrix(y_test, predictions2))

0.9993324432576769
Accuracy Score: 0.3959212376933896
Precision Score: 0.41851559737393806
Recall Score: 0.4116586697157502
[[129  31  18   9   5   2]
 [ 19  63  55  24  12   8]
 [  7  80  83  50  21  14]
 [  7  42  75  66  42  22]
 [  6  30  47  69  97  68]
 [  2   4  14  19  57 125]]


In [ ]:
y_test

In [10]:
pd.DataFrame({'Feature': X.columns, 'Importance': model.feature_importances_}).sort_values(by="Importance", ascending=False)

,Feature,Importance
0,Rating,0.197439
7,dias_sin_actualizar,0.130562
2,tamaño,0.118718
6,Genero1,0.072953
13,Genero2,0.070135
12,version_android,0.069731
1,Categoria,0.063229
8,meses_sin_actualizar,0.063124
3,tamaño2,0.042163
10,version_actual,0.038298


# RandomForest

#### Con Train Completo, dejamos PCA

In [46]:
pipe = Pipeline(steps=[("scaler", StandardScaler()),
                       ('pca', PCA(n_components=12)),
                       ('classifier', RandomForestClassifier())
])

rf_params = {
    "pca__n_components": randint(6,12),
    'scaler': [StandardScaler(), MinMaxScaler()],
    'classifier': [RandomForestClassifier()],
    'classifier__n_estimators': randint(100, 1000),
    'classifier__max_features': ['sqrt', 'log2', None],
    'classifier__max_depth': [None] + list(randint(3, 50).rvs(10)),
    'classifier__min_samples_split': randint(2, 21),
    'classifier__min_samples_leaf': randint(1, 21),
    'classifier__criterion': ['gini', 'entropy'],
    'classifier__max_leaf_nodes': [None] + list(randint(2, 50).rvs(10)),
    'classifier__min_impurity_decrease': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5],
    'classifier__class_weight': [None, 'balanced'],
    'classifier__random_state': [42],
    'classifier__warm_start': [True, False],
    'classifier__oob_score': [True, False]
}

search_space = [
    rf_params  ]

random_search_rf = RandomizedSearchCV(pipe,
                           search_space,
                           n_iter=500,
                           scoring='f1_macro', 
                           refit='f1_macro',
                           cv=5,
                           n_jobs=-1,
                           verbose=1,
                           random_state=42)


random_search_rf.fit(X_train, y_train)
print(random_search_rf.best_params_)
best_model_rf = random_search_rf.best_estimator_
predictions_rf = best_model_rf.predict(X_test)

print(best_model_rf.score(X_train, y_train))
print(accuracy_score(y_test, predictions_rf))
print(precision_score(y_test,predictions_rf, average="macro"))
print(recall_score(y_test, predictions_rf, average="macro"))
print(confusion_matrix(y_test, predictions_rf))

Fitting 5 folds for each of 500 candidates, totalling 2500 fits


c:\Users\javie\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_forest.py:862: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight ("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the resulting weights as the class_weight parameter.
  warn(


{'classifier': RandomForestClassifier(), 'classifier__class_weight': 'balanced', 'classifier__criterion': 'gini', 'classifier__max_depth': 13, 'classifier__max_features': None, 'classifier__max_leaf_nodes': None, 'classifier__min_impurity_decrease': 0.0, 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 14, 'classifier__n_estimators': 831, 'classifier__oob_score': False, 'classifier__random_state': 42, 'classifier__warm_start': True, 'pca__n_components': 10, 'scaler': MinMaxScaler()}
0.7556258790436006
0.4050632911392405
0.4286389035876205
0.41089765992882804
[[124  32  13  14   8   3]
 [ 23  51  56  26  19   6]
 [  4  73  88  57  25   8]
 [  7  31  70  73  57  16]
 [  3  20  44  63 127  60]
 [  0   3  10  21  74 113]]


#### Con el RandomUnderSampler, dejamos PCA

In [39]:
pipe = Pipeline(steps=[("scaler", StandardScaler()),
                       ('pca', PCA(n_components=12)),
                       ('classifier', RandomForestClassifier())
])

rf_params = {
    "pca__n_components": randint(6,12),
    'scaler': [StandardScaler(), MinMaxScaler()],
    'classifier': [RandomForestClassifier()],
    'classifier__n_estimators': randint(100, 1000),
    'classifier__max_features': ['sqrt', 'log2', None],
    'classifier__max_depth': [None] + list(randint(3, 50).rvs(10)),
    'classifier__min_samples_split': randint(2, 21),
    'classifier__min_samples_leaf': randint(1, 21),
    'classifier__criterion': ['gini', 'entropy'],
    'classifier__max_leaf_nodes': [None] + list(randint(2, 50).rvs(10)),
    'classifier__min_impurity_decrease': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5],
    'classifier__class_weight': [None, 'balanced'],
    'classifier__random_state': [42],
    'classifier__warm_start': [True, False],
    'classifier__oob_score': [True, False]
}

search_space = [
    rf_params  ]

random_search_rf2 = RandomizedSearchCV(pipe,
                           search_space,
                           n_iter=100,
                           scoring='f1_macro', 
                           refit='f1_macro',
                           cv=5,
                           n_jobs=-1,
                           verbose=1,
                           random_state=42)


random_search_rf2.fit(X_train_resampled, y_train_resampled)
print(random_search_rf2.best_params_)
best_model_rf2 = random_search_rf2.best_estimator_
predictions_rf2 = best_model_rf2.predict(X_test)

print(best_model_rf2.score(X_train_resampled, y_train_resampled))
print(accuracy_score(y_test, predictions_rf2))
print(precision_score(y_test,predictions_rf2, average="macro"))
print(recall_score(y_test, predictions_rf2, average="macro"))
print(confusion_matrix(y_test, predictions_rf2))

Fitting 5 folds for each of 100 candidates, totalling 500 fits
{'classifier': RandomForestClassifier(), 'classifier__class_weight': None, 'classifier__criterion': 'gini', 'classifier__max_depth': None, 'classifier__max_features': None, 'classifier__max_leaf_nodes': 24, 'classifier__min_impurity_decrease': 0.0, 'classifier__min_samples_leaf': 3, 'classifier__min_samples_split': 19, 'classifier__n_estimators': 892, 'classifier__oob_score': False, 'classifier__random_state': 42, 'classifier__warm_start': True, 'pca__n_components': 11, 'scaler': MinMaxScaler()}
0.46328437917222964
0.3818565400843882
0.40494159685750947
0.395421433846127
[[120  29  16  16   9   4]
 [ 22  53  51  26  24   5]
 [  3  60 100  44  31  17]
 [  4  25  93  50  50  32]
 [  1  10  60  49  90 107]
 [  0   0  18  17  56 130]]


In [49]:
best_model_rf2.fit(X,y)

c:\Users\javie\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_forest.py:468: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn(


Pipeline(steps=[('scaler', MinMaxScaler()), ('pca', PCA(n_components=11)),
                ('classifier',
                 RandomForestClassifier(max_features=None, max_leaf_nodes=24,
                                        min_samples_leaf=3,
                                        min_samples_split=19, n_estimators=892,
                                        random_state=42, warm_start=True))])

In [52]:
best_model_rf2.fit(X,y)
predictions_rf2_final = best_model_rf2.predict(final_test_x)
print(best_model_rf2.score(X, y))
print(accuracy_score(final_test_y, predictions_rf2_final))
print(precision_score(final_test_y,predictions_rf2_final, average="macro"))
print(recall_score(final_test_y, predictions_rf2_final, average="macro"))
print(confusion_matrix(final_test_y, predictions_rf2_final))

0.28621659634317864
0.23115860517435322
0.264297725949684
0.2388781214042995
[[ 30  89  89  32  12   4]
 [ 22  65  78  23  14   3]
 [ 25  64  92  31  32   5]
 [ 56  52  99  59  64  12]
 [ 98  35 114  52  99  46]
 [ 83  11  37  15  70  66]]


# XGB

In [47]:
pipe = Pipeline(steps=[("scaler", StandardScaler()),
                       ('pca', PCA(n_components=12)),
                       ('classifier', RandomForestClassifier())
])

xgb_params = {
    "pca__n_components": randint(6,12),
    "scaler": [StandardScaler(), MinMaxScaler()],
    'classifier': [XGBClassifier()],
    'classifier__n_estimators': randint(100, 1000),
    'classifier__max_features': ['auto', 'sqrt', 'log2'],
    'classifier__max_depth': [None] + list(randint(3, 50).rvs(10)),
    'classifier__min_samples_split': randint(2, 21),
    'classifier__min_samples_leaf': randint(1, 21),
    'classifier__bootstrap': [True, False],
    'classifier__criterion': ['gini', 'entropy'],
    'classifier__max_leaf_nodes': [None] + list(randint(2, 50).rvs(10)),
    'classifier__min_impurity_decrease': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5],
    'classifier__class_weight': [None, 'balanced'],
    'classifier__random_state': [42],
    'classifier__warm_start': [True, False],
    'classifier__oob_score': [True, False]
}

search_space = [
    xgb_params  ]

random_search_xgb = RandomizedSearchCV(pipe,
                           search_space,
                           n_iter=500,
                           scoring='f1_macro', 
                           refit='f1_macro',
                           cv=5,
                           n_jobs=-1,
                           verbose=1,
                           random_state=42)


random_search_xgb.fit(X_train, y_train)
print(random_search_xgb.best_params_)
best_model_xgb = random_search_xgb.best_estimator_
predictions_xgb = best_model_xgb.predict(X_test)

print(best_model_xgb.score(X_train, y_train))
print(accuracy_score(y_test, predictions_xgb))
print(precision_score(y_test,predictions_xgb, average="macro"))
print(recall_score(y_test, predictions_xgb, average="macro"))
print(confusion_matrix(y_test, predictions_xgb))

Fitting 5 folds for each of 500 candidates, totalling 2500 fits


c:\Users\javie\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [00:23:58] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "bootstrap", "class_weight", "criterion", "max_features", "max_leaf_nodes", "min_impurity_decrease", "min_samples_leaf", "min_samples_split", "oob_score", "warm_start" } are not used.

  warnings.warn(smsg, UserWarning)


{'classifier': XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...), 'classifier__bootstrap': False, 'classifier__class_weight': 'balanced', 'classifier__criterion': 'entropy', 'classifier__max_depth': 7, 'classifier__max_features': 'sqrt', 'classifier__max_leaf_nodes': 22, 'classifier__min_imp

In [53]:
best_model_xgb.fit(X,y)
predictions_xgb_final = best_model_xgb.predict(final_test_x)
print(best_model_xgb.score(X, y))
print(accuracy_score(final_test_y, predictions_xgb_final))
print(precision_score(final_test_y,predictions_xgb_final, average="macro"))
print(recall_score(final_test_y, predictions_xgb_final, average="macro"))
print(confusion_matrix(final_test_y, predictions_xgb_final))

c:\Users\javie\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [00:56:06] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "bootstrap", "class_weight", "criterion", "max_features", "max_leaf_nodes", "min_impurity_decrease", "min_samples_leaf", "min_samples_split", "oob_score", "warm_start" } are not used.

  warnings.warn(smsg, UserWarning)


0.9911392405063291
0.36051743532058494
0.38429675266724583
0.351648119979957
[[168  38  16  20  14   0]
 [ 16  55  60  35  33   6]
 [  9  63  55  60  54   8]
 [  8  38  68  80 124  24]
 [  3  32  42  98 211  58]
 [  0   6  14  35 155  72]]


In [48]:
pipe = Pipeline(steps=[("scaler", StandardScaler()),
                       ('pca', PCA(n_components=12)),
                       ('classifier', RandomForestClassifier())
])

xgb_params = {
    "pca__n_components": randint(6,12),
    "scaler": [StandardScaler(), MinMaxScaler()],
    'classifier': [XGBClassifier()],
    'classifier__n_estimators': randint(100, 1000),
    'classifier__max_features': ['auto', 'sqrt', 'log2'],
    'classifier__max_depth': [None] + list(randint(3, 50).rvs(10)),
    'classifier__min_samples_split': randint(2, 21),
    'classifier__min_samples_leaf': randint(1, 21),
    'classifier__bootstrap': [True, False],
    'classifier__criterion': ['gini', 'entropy'],
    'classifier__max_leaf_nodes': [None] + list(randint(2, 50).rvs(10)),
    'classifier__min_impurity_decrease': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5],
    'classifier__class_weight': [None, 'balanced'],
    'classifier__random_state': [42],
    'classifier__warm_start': [True, False],
    'classifier__oob_score': [True, False]
}

search_space = [
    xgb_params  ]

random_search_xgb2 = RandomizedSearchCV(pipe,
                           search_space,
                           n_iter=500,
                           scoring='f1_macro', 
                           refit='f1_macro',
                           cv=5,
                           n_jobs=-1,
                           verbose=1,
                           random_state=42)


random_search_xgb2.fit(X_train_resampled, y_train_resampled)
print(random_search_xgb2.best_params_)
best_model_xgb2 = random_search_xgb2.best_estimator_
predictions_xgb2 = best_model_xgb2.predict(X_test)

print(best_model_xgb2.score(X_train_resampled, y_train_resampled))
print(accuracy_score(y_test, predictions_xgb2))
print(precision_score(y_test,predictions_xgb2, average="macro"))
print(recall_score(y_test, predictions_xgb2, average="macro"))
print(confusion_matrix(y_test, predictions_xgb2))

Fitting 5 folds for each of 500 candidates, totalling 2500 fits


c:\Users\javie\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [00:44:52] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "bootstrap", "criterion", "max_features", "max_leaf_nodes", "min_impurity_decrease", "min_samples_leaf", "min_samples_split", "oob_score", "warm_start" } are not used.

  warnings.warn(smsg, UserWarning)


{'classifier': XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...), 'classifier__bootstrap': True, 'classifier__class_weight': None, 'classifier__criterion': 'entropy', 'classifier__max_depth': 21, 'classifier__max_features': 'sqrt', 'classifier__max_leaf_nodes': 22, 'classifier__min_impurity_

In [54]:
best_model_xgb2.fit(X,y)
predictions_xgb2_final = best_model_xgb2.predict(final_test_x)
print(best_model_xgb2.score(X, y))
print(accuracy_score(final_test_y, predictions_xgb2_final))
print(precision_score(final_test_y,predictions_xgb2_final, average="macro"))
print(recall_score(final_test_y, predictions_xgb2_final, average="macro"))
print(confusion_matrix(final_test_y, predictions_xgb2_final))

c:\Users\javie\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [00:57:34] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "bootstrap", "criterion", "max_features", "max_leaf_nodes", "min_impurity_decrease", "min_samples_leaf", "min_samples_split", "oob_score", "warm_start" } are not used.

  warnings.warn(smsg, UserWarning)


0.9954992967651195
0.3683914510686164
0.38714965837462034
0.3631017735081398
[[167  41  20  20   7   1]
 [ 23  56  59  35  32   0]
 [ 12  60  67  48  52  10]
 [  6  43  63  87 112  31]
 [  2  22  56  93 198  73]
 [  1   5  17  35 144  80]]


In [44]:
'''import pickle

filename = 'models/xgb_+rating_MS099_ACC39_PS41_RS40_.pkl'

with open(filename, 'wb') as archivo_salida:
    pickle.dump(best_model_xgb2, archivo_salida)'''

# Pipeline con varios modelos para ver si mejoramos el 0.4

In [56]:
pipe_multi = Pipeline(steps=[("scaler", StandardScaler()),
                             ('pca', PCA(n_components=12)),
                             ('classifier', RandomForestClassifier())
])

xgb_params = {
    "pca__n_components": randint(6,12),
    "scaler": [StandardScaler(), MinMaxScaler()],
    'classifier': [XGBClassifier()],
    'classifier__n_estimators': randint(50, 500),
    "classifier__learning_rate": [0.001, 0.01, 0.1, 0.5, 1.0],
    "classifier__max_depth": [3, 4, 5, 6, 7],
    "classifier__min_child_weight": randint(1, 11),
    "classifier__subsample": [0.5, 0.7, 0.9, 1.0],
    "classifier__colsample_bytree": [0.5, 0.7, 0.9, 1.0]
    
}

knn_params = {
    "pca__n_components": randint(6,12),
    "scaler": [StandardScaler(), MinMaxScaler()],
    'classifier': [KNeighborsClassifier()],
    'classifier__n_neighbors':randint(1, 20),
    'classifier__weights': ['uniform', 'distance'],
    'classifier__p': [1, 2]
}


gb_params = {
    "pca__n_components": randint(6,12),
    "scaler": [StandardScaler(), MinMaxScaler()],
    'classifier': [GradientBoostingClassifier()],
    'classifier__n_estimators': randint(50, 500),
    'classifier__learning_rate': [0.001, 0.01, 0.1, 0.5, 1.0],
    'classifier__max_depth': [3, 4, 5, 6, 7],
    'classifier__min_samples_split': randint(2, 11),
    'classifier__min_samples_leaf': randint(1, 11),
    'classifier__subsample': [0.5, 0.7, 0.9, 1.0]
}

search_space_multi = [
    xgb_params,
    knn_params,
    gb_params  ]


rsm_multi = RandomizedSearchCV(pipe_multi,
                           search_space_multi,
                           n_iter=200,
                           scoring='f1_macro', 
                           refit='f1_macro',
                           cv=5,
                           n_jobs=-1,
                           verbose=1,
                           random_state=42)

rsm_multi.fit(X_train, y_train)
print(rsm_multi.best_params_)
best_model_multi = rsm_multi.best_estimator_
predictions_multi = best_model_multi.predict(X_test)

print(best_model_multi.score(X_train, y_train))
print(accuracy_score(y_test, predictions_multi))
print(precision_score(y_test, predictions_multi, average="macro"))
print(recall_score(y_test, predictions_multi, average="macro"))
print(confusion_matrix(y_test, predictions_multi))

Fitting 5 folds for each of 200 candidates, totalling 1000 fits
{'classifier': XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...), 'classifier__colsample_bytree': 0.9, 'classifier__learning_rate': 0.01, 'classifier__max_depth': 7, 'classifier__min_child_weight': 1, 'classifier__n_estimators':

In [58]:
best_model_multi.fit(X,y)
predictions_multi_final = best_model_multi.predict(final_test_x)
print(best_model_multi.score(X, y))
print(accuracy_score(final_test_y, predictions_multi_final))
print(precision_score(final_test_y,predictions_multi_final, average="macro"))
print(recall_score(final_test_y, predictions_multi_final, average="macro"))
print(confusion_matrix(final_test_y, predictions_multi_final))

0.6954992967651196
0.4032620922384702
0.4337693158328207
0.38291928638005696
[[167  31  24  20  13   1]
 [ 16  45  75  27  41   1]
 [  8  45  90  35  67   4]
 [  4  29  78  58 157  16]
 [  3  12  54  52 287  36]
 [  0   1  12  12 187  70]]


In [57]:
pipe_multi = Pipeline(steps=[("scaler", StandardScaler()),
                             ('pca', PCA(n_components=12)),
                             ('classifier', RandomForestClassifier())
])

xgb_params = {
    "pca__n_components": randint(6,12),
    "scaler": [StandardScaler(), MinMaxScaler()],
    'classifier': [XGBClassifier()],
    'classifier__n_estimators': randint(50, 500),
    "classifier__learning_rate": [0.001, 0.01, 0.1, 0.5, 1.0],
    "classifier__max_depth": [3, 4, 5, 6, 7],
    "classifier__min_child_weight": randint(1, 11),
    "classifier__subsample": [0.5, 0.7, 0.9, 1.0],
    "classifier__colsample_bytree": [0.5, 0.7, 0.9, 1.0]
    
}

knn_params = {
    "pca__n_components": randint(6,12),
    "scaler": [StandardScaler(), MinMaxScaler()],
    'classifier': [KNeighborsClassifier()],
    'classifier__n_neighbors':randint(1, 20),
    'classifier__weights': ['uniform', 'distance'],
    'classifier__p': [1, 2]
}


gb_params = {
    "pca__n_components": randint(6,12),
    "scaler": [StandardScaler(), MinMaxScaler()],
    'classifier': [GradientBoostingClassifier()],
    'classifier__n_estimators': randint(50, 500),
    'classifier__learning_rate': [0.001, 0.01, 0.1, 0.5, 1.0],
    'classifier__max_depth': [3, 4, 5, 6, 7],
    'classifier__min_samples_split': randint(2, 11),
    'classifier__min_samples_leaf': randint(1, 11),
    'classifier__subsample': [0.5, 0.7, 0.9, 1.0]
}

search_space_multi2 = [
    xgb_params,
    knn_params,
    gb_params  ]


rsm_multi2 = RandomizedSearchCV(pipe_multi,
                           search_space_multi2,
                           n_iter=200,
                           scoring='f1_macro', 
                           refit='f1_macro',
                           cv=5,
                           n_jobs=-1,
                           verbose=1,
                           random_state=42)

rsm_multi2.fit(X_train_resampled, y_train_resampled)
print(rsm_multi2.best_params_)
best_model_multi2 = rsm_multi2.best_estimator_
predictions_multi2 = best_model_multi2.predict(X_test)

print(best_model_multi2.score(X_train_resampled, y_train_resampled))
print(accuracy_score(y_test, predictions_multi2))
print(precision_score(y_test, predictions_multi2, average="macro"))
print(recall_score(y_test, predictions_multi2, average="macro"))
print(confusion_matrix(y_test, predictions_multi2))

Fitting 5 folds for each of 200 candidates, totalling 1000 fits
{'classifier': XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...), 'classifier__colsample_bytree': 0.9, 'classifier__learning_rate': 0.01, 'classifier__max_depth': 7, 'classifier__min_child_weight': 1, 'classifier__n_estimators':

In [60]:
import pickle

filename = 'models/xgb_+rating_MS078_ACC42_PS44_RS43_.pkl'

with open(filename, 'wb') as archivo_salida:
    pickle.dump(best_model_multi2, archivo_salida)

In [ ]:
'''{'classifier': XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...), 'classifier__colsample_bytree': 0.9, 
              'classifier__learning_rate': 0.01, 'classifier__max_depth': 7, 'classifier__min_child_weight': 1,
                'classifier__n_estimators': 459, 'classifier__subsample': 0.5, 'pca__n_components': 9, 'scaler': MinMaxScaler()}'''

In [59]:
best_model_multi2.fit(X,y)
predictions_multi_final2 = best_model_multi2.predict(final_test_x)
print(best_model_multi2.score(X, y))
print(accuracy_score(final_test_y, predictions_multi_final2))
print(precision_score(final_test_y,predictions_multi_final2, average="macro"))
print(recall_score(final_test_y, predictions_multi_final2, average="macro"))
print(confusion_matrix(final_test_y, predictions_multi_final2))

0.6954992967651196
0.4032620922384702
0.4337693158328207
0.38291928638005696
[[167  31  24  20  13   1]
 [ 16  45  75  27  41   1]
 [  8  45  90  35  67   4]
 [  4  29  78  58 157  16]
 [  3  12  54  52 287  36]
 [  0   1  12  12 187  70]]
